# 001-Using Auto Whenever Possible
## How to do it...

* To declare local variables with the form auto name = expression when you do not want to commit to a specific type:

In [2]:
#include <iostream>

auto i = 42;          // int 
auto d = 42.5;        // double 
auto s = "text";      // char const * 
auto v = { 1, 2, 3 }; // std::initializer_list<int>
std::cout << s << std::endl;

text


* To declare local variables with the auto name = type-id { expression } form when you need to commit to a specific type:

In [3]:
auto b  = new char[10]{ 0 };            // char*
auto s1 = std::string {"text"};         // std::string
auto v1 = std::vector<int> { 1, 2, 3 }; // std::vector<int>
auto p  = std::make_shared<int>(42);    // std::shared_ptr<int>
std::cout << *p << std::endl;

42


* To declare named lambda functions, with the form auto name = lambda-expression, unless the lambda needs to be passed or return to a function:

In [4]:
auto upper = [](char const c) {return toupper(c); };
std::cout << char(upper('x')) << std::endl;

X


* To declare lambda parameters and return values:

In [5]:
auto add = [](auto const a, auto const b) {return a + b;};
std::cout << add(3, 1.2) << std::endl;

4.2


* To declare function return type when you don't want to commit to a specific type:

In [6]:
template <typename F, typename T> 
auto apply(F&& f, T value) 
{ 
    return f(value);
}



## How it works...

The auto specifier is basically a placeholder for an actual type. When using auto, the compiler deduces the actual type from the following instances:

From the **type of the expression used to initialize a variable**, when auto is used to declare variables.

From the trailing **return type or the type of the return expression of a function**, when auto is used as a placeholder for the return type of a function.

#### Benefits of using auto specifier

* It's not possible to leave a variable unitialized
* Using auto ensures that you always use the correct type and that implicit converion will not occur.

  For example: Consider the following example where we retrieve the size of a vector to a local variable. In the first case, the type of the variable is int, though the size() method returns size_t. That means an implicit conversion from size_t to int will occur. However, using auto for the type will deduce the correct type, that is, size_t:

In [7]:
auto v = std::vector<int>{ 1, 2, 3 }; 
int size1 = v.size();       
// implicit conversion, possible loss of data 
auto size2 = v.size(); 
auto size3 = int{ v.size() };  // error, narrowing conversion

input_line_14:6:19: error: non-constant-expression cannot be narrowed from type 'std::vector::size_type' (aka 'unsigned long') to 'int' in initializer list [-Wc++11-narrowing]
auto size3 = int{ v.size() };  // error, narrowing conversion
                  ^~~~~~~~
input_line_14:6:19: note: insert an explicit cast to silence this issue
auto size3 = int{ v.size() };  // error, narrowing conversion
                  ^~~~~~~~
                  static_cast<int>( )


Interpreter Error: 

* Using auto promotes good object-oriented practices, such as preferring interfaces over implementations. **The lesser the number of types specified the more generic the code is and more open to future changes**, which is a fundamental principle of object-oriented programming.

* When you want to iterate over a range, you don't care about the actual type of the iterator. You are only interested in the iterator itself; so, using auto saves time used for typing possibly long names and helps you focus on actual code and not type names.

In [8]:
std::map<int, std::string> m; 
for (std::map<int,std::string>::const_iterator it = m.cbegin();
it != m.cend(); ++it) 
{ /*...*/ } 

for (auto it = m.cbegin(); it != m.cend(); ++it)
{ /*...*/ }

* Declaring variables with auto provides a consistent coding style with the type always in the right-hand side. If you allocate objects dynamically, you need to write the type both on the left and right side of the assignment, for example, int p = new int(42). With auto, the type is specified only once on the right side.

#### Some gotchas when using auto

* The auto specifier is only a placeholder for the type, not for the const/volatile and references specifiers. **If you need a const/volatile and/or reference type, then you need to specify them explicitly**. In the following example, foo.get() returns a reference to int; when variable x is initialized from the return value, the type deduced by the compiler is int, and not int&. Therefore, any change to x will not propagate to foo.x_. In order to do so, one should use auto&:

In [9]:
class foo { 
          int x_; 
        public: 
          foo(int const x = 0) :x_{ x } {} 
          int& get() { return x_; } 
        }; 

        foo f(42); 
        auto x = f.get(); 
        x = 100; 
        std::cout << f.get() << std::endl; // prints 42

42


@0x7fea11fcab60

* It is not possible to use auto for types that are not moveable:


In [10]:
auto ai = std::atomic<int>(42); // error

input_line_20:2:12: error: implicit instantiation of undefined template 'std::atomic<int>'
 auto ai = std::atomic<int>(42); // error
           ^
/../lib/gcc/x86_64-linux-gnu/9/../../../../include/c++/9/bits/atomic_base.h:145:12: note: template is declared here
    struct atomic;
           ^


Interpreter Error: 

* It is not possible to use auto for multi-word types, such as long long, long double, or struct foo. However, in the first case, the possible workarounds are to use literals or type aliases; as for the second, using struct/class in that form is only supported in C++ for C compatibility and should be avoided anyways:

In [11]:
using llong = long long;
auto l2 = llong{ 42 };     // OK 
auto l3 = 42LL;            // OK
auto l1 = long long{ 42 }; // error 

input_line_21:5:16: error: expected '(' for function-style cast or type construction
auto l1 = long long{ 42 }; // error 
          ~~~~ ^


Interpreter Error: 

* If you use the auto specifier but still need to know the type, you can do so in any IDE by putting the cursor over a variable, for instance. If you leave the IDE, however, that is not possible anymore, and the only way to know the actual type is to deduce it yourself from the initialization expression, which could probably mean searching through the code for function return types.

* The auto can be used to specify the return type from a function. In C++11, this requires a trailing return type in the function declaration. In C++14, this has been relaxed, and the type of the return value is deduced by the compiler from the return expression. If there are multiple return values they should have the same type:

In [22]:
// C++11 
auto func1(int const i) -> int
{ return 2*i; }

input_line_32:4:1: error: function definition is not allowed here
{ return 2*i; }
^


Interpreter Error: 

In [17]:
// C++14 
auto func2(int const i) 
{ return 2*i; }

* As mentioned earlier, auto does not retain const/volatile and reference qualifiers. This leads to problems with auto as a placeholder for the return type from a function. To explain this, let us consider the preceding example with foo.get(). This time we have a wrapper function called proxy_get() that takes a reference to a foo, calls get(), and returns the value returned by get(), which is an int&. However, the compiler will deduce the return type of proxy_get() as being int, not int&. Trying to assign that value to an int& fails with an error:

In [36]:
class foo 
    { 
      int x_; 
    public: 
      foo(int const x = 0) :x_{ x } {} 
      int& get() { return x_; } 
    }; 

auto proxy_get(foo& f) { return f.get(); } 

auto f = foo{ 42 }; 
auto& x = proxy_get(f); // cannot convert from 'int' to 'int &'

input_line_46:9:25: error: function definition is not allowed here
 auto proxy_get(foo& f) { return f.get(); } 
                        ^
input_line_46:11:11: error: use of undeclared identifier 'proxy_get'
auto& x = proxy_get(f); // cannot convert from 'int' to 'int &'
          ^


Interpreter Error: 

To fix this, we need to actually return auto&. However, this is a problem with templates and perfect forwarding the return type without knowing whether that is a value or a reference. The solution to this problem in C++14 is decltype(auto) that will correctly deduce the type:

<iframe width="800px" height="200px" src="https://godbolt.org/e#g:!((g:!((g:!((h:codeEditor,i:(fontScale:12,j:1,lang:c%2B%2B,selection:(endColumn:68,endLineNumber:13,positionColumn:68,positionLineNumber:13,selectionStartColumn:68,selectionStartLineNumber:13,startColumn:68,startLineNumber:13),source:'class+foo+%0A++++%7B+%0A++++++int+x_%3B+%0A++++public:+%0A++++++foo(int+const+x+%3D+0)+:x_%7B+x+%7D+%7B%7D+%0A++++++int%26+get()+%7B+return+x_%3B+%7D+%0A++++%7D%3B+%0A%0A++++//auto+proxy_get(foo%26+f)+%7B+return+f.get()%3B+%7D+//auto+cannot+deduce+reference+type+so+its+an+int%0A++++decltype(auto)+proxy_get(foo%26+f)+%7B+return+f.get()%3B+%7D+%0A%0A++++auto+f+%3D+foo%7B+42+%7D%3B+%0A++++auto%26+x+%3D+proxy_get(f)%3B+//+cannot+convert+from+!'int!'+to+!'int+%26!''),l:'5',n:'0',o:'C%2B%2B+source+%231',t:'0')),k:50,l:'4',n:'0',o:'',s:0,t:'0'),(g:!((g:!((h:compiler,i:(compiler:g102,filters:(b:'0',binary:'1',commentOnly:'0',demangle:'0',directives:'0',execute:'0',intel:'0',libraryCode:'1',trim:'1'),fontScale:14,j:1,lang:c%2B%2B,libs:!(),options:'',selection:(endColumn:1,endLineNumber:1,positionColumn:1,positionLineNumber:1,selectionStartColumn:1,selectionStartLineNumber:1,startColumn:1,startLineNumber:1),source:1),l:'5',n:'0',o:'x86-64+gcc+10.2+(Editor+%231,+Compiler+%231)+C%2B%2B',t:'0')),k:50,l:'4',m:50,n:'0',o:'',s:0,t:'0'),(g:!((h:output,i:(compiler:1,editor:1,fontScale:14,wrap:'1'),l:'5',n:'0',o:'%231+with+x86-64+gcc+10.2',t:'0')),header:(),l:'4',m:50,n:'0',o:'',s:0,t:'0')),k:50,l:'3',n:'0',o:'',t:'0')),l:'2',n:'0',o:'',t:'0')),version:4"></iframe>

<iframe width="800px" height="200px" src="https://godbolt.org/e#g:!((g:!((g:!((h:codeEditor,i:(fontScale:12,j:1,lang:c%2B%2B,selection:(endColumn:68,endLineNumber:13,positionColumn:68,positionLineNumber:13,selectionStartColumn:68,selectionStartLineNumber:13,startColumn:68,startLineNumber:13),source:'class+foo+%0A++++%7B+%0A++++++int+x_%3B+%0A++++public:+%0A++++++foo(int+const+x+%3D+0)+:x_%7B+x+%7D+%7B%7D+%0A++++++int%26+get()+%7B+return+x_%3B+%7D+%0A++++%7D%3B+%0A%0A++++//auto+proxy_get(foo%26+f)+%7B+return+f.get()%3B+%7D+//auto+cannot+deduce+reference+type+so+its+an+int%0A++++decltype(auto)+proxy_get(foo%26+f)+%7B+return+f.get()%3B+%7D+%0A%0A++++auto+f+%3D+foo%7B+42+%7D%3B+%0A++++auto%26+x+%3D+proxy_get(f)%3B+//+cannot+convert+from+!'int!'+to+!'int+%26!''),l:'5',n:'0',o:'C%2B%2B+source+%231',t:'0')),k:50,l:'4',n:'0',o:'',s:0,t:'0'),(g:!((g:!((h:compiler,i:(compiler:g102,filters:(b:'0',binary:'1',commentOnly:'0',demangle:'0',directives:'0',execute:'0',intel:'0',libraryCode:'1',trim:'1'),fontScale:14,j:1,lang:c%2B%2B,libs:!(),options:'',selection:(endColumn:1,endLineNumber:1,positionColumn:1,positionLineNumber:1,selectionStartColumn:1,selectionStartLineNumber:1,startColumn:1,startLineNumber:1),source:1),l:'5',n:'0',o:'x86-64+gcc+10.2+(Editor+%231,+Compiler+%231)+C%2B%2B',t:'0')),k:50,l:'4',m:50,n:'0',o:'',s:0,t:'0'),(g:!((h:output,i:(compiler:1,editor:1,fontScale:14,wrap:'1'),l:'5',n:'0',o:'%231+with+x86-64+gcc+10.2',t:'0')),header:(),l:'4',m:50,n:'0',o:'',s:0,t:'0')),k:50,l:'3',n:'0',o:'',t:'0')),l:'2',n:'0',o:'',t:'0')),version:4"></iframe>

````c++
class foo 
    { 
      int x_; 
    public: 
      foo(int const x = 0) :x_{ x } {} 
      int& get() { return x_; } 
    }; 

auto proxy_get(foo& f) { return f.get(); } //auto cannot deduce reference type so its an int
decltype(auto) proxy_get(foo& f) { return f.get(); } //The solution to this problem in C++14 is decltype(auto) that will correctly deduce the type

auto f = foo{ 42 }; 
decltype(auto) x = proxy_get(f); // cannot convert from 'int' to 'int &'
````

* The last important case where auto can be used is with lambdas. As of C++14, both lambda return type and lambda parameter types can be auto. Such a lambda is called a generic lambda because the closure type defined by the lambda has a templated call operator. The following shows a generic lambda that takes two auto parameters and returns the result of applying operator+ on the actual types:
````c++
    auto ladd = [] (auto const a, auto const b) { return a + b; }; 
    struct 
    { 
       template<typename T, typename U> 
       auto operator () (T const a, U const b) const { return a+b; } 
    } L;
````
This lambda can be used to add anything for which the operator+ is defined. In the following example, we use the lambda to add two integers and to concatenate to std::string objects (using the C++14 user-defined literal operator ""s):
````c++
    auto i = ladd(40, 2);            // 42 
    auto s = ladd("forty"s, "two"s); // "fortytwo"s
`````